In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
#Other model will go here as well
import LACER as lc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sgcrf import SparseGaussianCRF

def preprocessing(df, start_date, end_date):
    """
    Filters dataframe by specified start and end_dates and runs CleanedFrame on it.  
    """ 

    #Filter dataframe by dates 
    df = df[(df['Just Date'] >= start_date) & (df['Just Date'] <= end_date)]
    df = lc.CleanedFrame(df)

    return df
def lacer(df, df1, train_start_date, train_end_date, test_start_date, test_end_date, request_type, CD, predictor_num): #Once model is ready, replace df with csv
    """
    Trains 3 GCRF models on data from specified CD, Request Type, and Owner which is assigned to fulfill request. 
    Uses specified start and end dates for training and testing to creat train and test sets. 
    """

    #Create Training and Testing Sets
    dftrain = preprocessing(df , train_start_date, train_end_date)
    dftrain = dftrain.reset_index(drop = True)
    dftest = preprocessing(df1, test_start_date, test_end_date)
    dftest = dftest.reset_index(drop = True)

    #Reserve test set for training on all 3 models. 
    y_train, y_test = lc.CreateTestSet(dftest, predictor_num)
    y_test = y_test.reshape((-1, 1))


## 2 Models
    #Model1: CD
    modelCD = SparseGaussianCRF(lamL=0.1, lamT=0.1, n_iter=10000)
    dftrainCD = dftrain[dftrain['CD'] == CD].reset_index(drop = True)

    X_trainCD, X_testCD = lc.CreateTrainSet(dftrainCD, predictor_num)
    X_testCD = X_testCD.reshape((-1, 1))
    modelCD.fit(X_trainCD, X_testCD)

    y_predCD = modelCD.predict(y_train)

    #Model2: Request_type
    modelRT = SparseGaussianCRF(lamL=0.1, lamT=0.1, n_iter=10000)
    dftrainRT = dftrain[dftrain['RequestType'] == request_type].reset_index(drop = True)

    X_trainRT, X_testRT = lc.CreateTrainSet(dftrainRT, predictor_num)
    X_testRT = X_testRT.reshape((-1, 1))

    modelRT.fit(X_trainRT, X_testRT)

    y_predRT = modelRT.predict(y_train)


    #Average out all predictions
    y_predFinal = (y_predCD + y_predRT )/2

    #Return metrics 
    return lc.metrics(y_predFinal, y_test)

def gelev(val): 
    """
    Records whether or not a number is greater than 7. 
    """
    if val <= 11: 
        return 0
    else: 
        return 1
    
def preprocess(df_path):
    df = pd.read_csv(df_path)
    df['Just Date'] = df['Just Date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
    df['Eleven'] = df['ElapsedDays'].apply(gelev, 0)
    #Encode values
    c = ['Anonymous','AssignTo', 'RequestType', 'RequestSource','CD','Direction', 'ActionTaken', 'APC' ,'AddressVerified']
    d = ['Latitude', 'Longitude']
    #Put desired columns into dataframe, drop nulls. 
    dfn = df.filter(items = c + d + ['ElapsedDays'])
    dfn = dfn.dropna()
    #Separate data into explanatory and response variables
    XCAT = dfn.filter(items = c).values
    XNUM = dfn.filter(items = d).values
    y = dfn['ElapsedDays'] <= 11
    #Encode cateogrical data and merge with numerical data
    labelencoder_X = LabelEncoder()
    for num in range(len(c)): 
        XCAT[:, num] = labelencoder_X.fit_transform(XCAT[:, num])
    onehotencoder = OneHotEncoder()
    XCAT = onehotencoder.fit_transform(XCAT).toarray()
    X = np.concatenate((XCAT, XNUM), axis=1)
    return X,y, dfn

#50,20
def estimation_model(estimators, depth,X,y):    
    rf = RandomForestClassifier(n_estimators = estimators, max_depth = depth)
    print('creating train, test, val split')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)
    #Train Model
    print('training model')
    rf.fit(X_train, y_train)
    #Test model
    print('testing model')
    y_vpred = rf.predict(X_val)
    #Print Accuracy Function results
    print("Accuracy:",metrics.accuracy_score(y_val, y_vpred))
    print("Precision, Recall, F1Score:",metrics.precision_recall_fscore_support(y_val, y_vpred, average = 'binary'))
    return rf
    
def dummy_model(df):
    return 0,0

def split_to_models(df_file_path):
    print('Calculating train data and labels')
    X, y, df = preprocess(df_file_path)
    df_clean = pd.read_csv(df_file_path)
    print('Creating 11 day classifier')
    model_eleven = estimation_model(50,20,X,y)
    df['LessEqualEleven'] = model_eleven.predict(X)
    df['LessEqualEleven'] = df['LessEqualEleven'].apply(lambda x: int(x))
    df_sgcrf = df[df['LessEqualEleven'] == 1.0]
    df_other = df[df['LessEqualEleven'] == 0.0]
    return df_sgcrf.merge(df_clean,on=['Latitude','Longitude']), df_other

def run_split_models(df_file_path,train_start_date, train_end_date, test_start_date, test_end_date, request_type, CD, predictor_num):
    df_sgcrf, df_other = split_to_models(df_file_path)
    print('running SGCRF')
    rmse, mae = lacer(df_sgcrf.copy(),df_sgcrf.copy(), train_start_date, train_end_date, test_start_date, test_end_date, request_type, CD, predictor_num)
    print('running (other model)')
    a,b = dummy_model(df_other)
    return (rmse,mae),(a,b)

In [3]:
u,v = run_split_models('fservice.csv',"2018-01-01", "2018-06-01", "2018-06-02", "2018-09-02",'Graffiti Removal',5,50)

Calculating train data and labels


KeyboardInterrupt: 

In [3]:
import EDPipeline as edp

In [4]:
edp.run_split_models(pd.read_csv('fservice.csv'),"2018-01-01", "2018-06-01", "2018-06-02", "2018-09-02",'Graffiti Removal',5,50)

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Calculating train data and labels


C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Creating 11 day classifier
creating train, test, val split
training model
testing model
Accuracy: 0.9627461717578341
Precision, Recall, F1Score: (0.969112941828534, 0.992663471935135, 0.9807468484007124, None)
running SGCRF


C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ElapsedHours'] = df.apply(lambda x: elapsedHours(x['CreatedDate'],x['ClosedDate']),axis=1)
C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ClosedDate'] = df['ClosedDate'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

running (other model)


In [11]:
xx,yy,dff = edp.preprocess(pd.read_csv('fservice.csv'))

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [13]:
dff.head()

,Anonymous,AssignTo,RequestType,RequestSource,CD,Direction,ActionTaken,APC,AddressVerified,Latitude,Longitude,ElapsedDays,Just Date,CreatedDate,ClosedDate
1,Y,CCAC,Graffiti Removal,Mobile App,13.0,N,SR Created,East Los Angeles APC,Y,34.084714,-118.260479,11,2017-01-01,01/01/2017 12:45:00 AM,01/12/2017 05:10:00 PM
3,N,WVA,Graffiti Removal,Mobile App,12.0,W,SR Created,North Valley APC,Y,34.256993,-118.604730,3,2017-01-01,01/01/2017 01:01:00 AM,01/04/2017 05:33:00 PM
4,N,WVA,Graffiti Removal,Mobile App,12.0,W,SR Created,North Valley APC,Y,34.257061,-118.594999,3,2017-01-01,01/01/2017 01:09:00 AM,01/04/2017 05:34:00 PM
5,Y,NEGB_ELA,Graffiti Removal,Mobile App,1.0,E,SR Created,East Los Angeles APC,Y,34.090852,-118.226214,2,2017-01-01,01/01/2017 01:16:00 AM,01/03/2017 06:55:00 PM
6,N,NC,Bulky Items,Mobile App,4.0,N,SR Created,Central APC,Y,34.084822,-118.333307,6,2017-01-01,01/01/2017 01:21:00 AM,01/07/2017 09:46:00 AM


In [12]:
df_sgcrf, df_other = edp.split_to_models(pd.read_csv('fservice.csv'))

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Calculating train data and labels


C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Creating 11 day classifier
creating train, test, val split
training model
testing model
Accuracy: 0.9626080393085484
Precision, Recall, F1Score: (0.9688746625297175, 0.9927744359544067, 0.9806789576655174, None)


In [14]:
df_sgcrf.head()

,Anonymous,AssignTo,RequestType,RequestSource,CD,Direction,ActionTaken,APC,AddressVerified,Latitude,Longitude,ElapsedDays,Just Date,CreatedDate,ClosedDate,LessEqualEleven
1,Y,CCAC,Graffiti Removal,Mobile App,13.0,N,SR Created,East Los Angeles APC,Y,34.084714,-118.260479,11,2017-01-01,01/01/2017 12:45:00 AM,01/12/2017 05:10:00 PM,1
3,N,WVA,Graffiti Removal,Mobile App,12.0,W,SR Created,North Valley APC,Y,34.256993,-118.604730,3,2017-01-01,01/01/2017 01:01:00 AM,01/04/2017 05:33:00 PM,1
4,N,WVA,Graffiti Removal,Mobile App,12.0,W,SR Created,North Valley APC,Y,34.257061,-118.594999,3,2017-01-01,01/01/2017 01:09:00 AM,01/04/2017 05:34:00 PM,1
5,Y,NEGB_ELA,Graffiti Removal,Mobile App,1.0,E,SR Created,East Los Angeles APC,Y,34.090852,-118.226214,2,2017-01-01,01/01/2017 01:16:00 AM,01/03/2017 06:55:00 PM,1
6,N,NC,Bulky Items,Mobile App,4.0,N,SR Created,Central APC,Y,34.084822,-118.333307,6,2017-01-01,01/01/2017 01:21:00 AM,01/07/2017 09:46:00 AM,1


In [ ]:
dfa = preprocessing(df_sgcrf.copy(), "2018-06-02", "2018-09-02")

In [ ]:
dfa = dfa.reset_index(drop=True)
dfa.head()

In [18]:
y_train = []
y_test = []
predictor_num=50
#Create testing Set 
for i in range(0,dfa.shape[0] - predictor_num): 
    y_train.append(dfa['ElapsedHours'][i:i + predictor_num].values)
    y_test.append(dfa['ElapsedHours'][i + predictor_num])

In [ ]:
import pickle
modelCD = pickle.load(open('modelCD.pkl', 'rb'))
modelRT = pickle.load(open('modelRT.pkl', 'rb'))

int_features = [int(x) for x in request.form.values()]
final = [np.array(int_features)]
print(int_features)
print(final)
# averaged prediction --- pulled out from lacer script
prediction = modelCD.predict(final) + modelRT.predict(final) / 2
#return render_template('index.html', pred=str(prediction))